<a href="https://colab.research.google.com/github/okana2ki/intro-to-AI/blob/main/image_recognition_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import urllib.request
import torch
from torchvision import models, transforms
from PIL import Image

# ImageNetのクラス名をダウンロードします
class_idx = json.load(urllib.request.urlopen('https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'))

# 学習済みのResNet50モデルをロードします
resnet = models.resnet50(pretrained=True)
resnet.eval()

# 入力画像を処理するための変換を定義します
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

while True:
    # ユーザーに画像URLを入力させます
    img_url = input("画像のURLを入力してください（終了するには'q'を入力）：")

    if img_url.lower() == 'q':
        break

    # URLから画像をダウンロードし、メモリ内で開きます
    with urllib.request.urlopen(img_url) as url:
        img = Image.open(url)

    # 画像を変換します
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)

    # モデルを使って画像を推測します
    resnet.eval()
    out = resnet(batch_t)

    # ソフトマックス関数を適用して確率を計算します
    probs = torch.nn.functional.softmax(out, dim=1)[0] * 100

    # 上位3つの予測結果を取得します
    _, indices = torch.topk(probs, 3)
    percentages = probs[indices]

    # 推測結果を表示します
    for idx, percentage in zip(indices, percentages):
        print("クラス:", class_idx[idx.item()], ", 確信度:", percentage.item())

    print('\n' + '=' * 50 + '\n')

ファインチューニングの練習　2024/4/30

ファインチューニング用のデータの準備

In [ ]:
from tensorflow.keras.datasets import cifar10

# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize the pixel values
train_images = train_images / 255.0
test_images = test_images / 255.0

print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

事前学習モデルをロードして、ファインチューニング

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

# 1. Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# 2. Freeze the convolutional layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# 3. Add new fully connected layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# 4. Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# 5. Load the training and validation data
# train_data = ...
# validation_data = ...

# 6. Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 7. Train the model
# model.fit(train_data, validation_data=validation_data, epochs=5)
model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

# 8. Save the model
model.save('fine_tuned_model.h5')